In [1]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump
from copy import deepcopy

sys.path.append('../../.')
import pIRPgym

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [60,80]}                                           # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [80,120]}                                          # Purchase costs at suppliers'

d_params = {'dist': 'log-normal-trunc', 'r_f_params': [3,1], 'break':104.5, 'offset':15}          # Demand
h_params = {'dist': 'd_uniform', 'r_f_params': [20,40]}                                           # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Experiments

In [2]:
T = 7; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; theta = 0.95; gamma = 1

env_config = {'M':5,'K':2,'T':T,'F':2,'Q':375,'S':3,'LA_horizon':4,'d_max':2000,'hist_window':80, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]; seed_ix = 0
objectives = ["costs","climate","water","land","fossil"]

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [ ]:
for stoch_seed in feas_seeds:
    for w in weights:

        rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

        stoch_rd_seed = stoch_seed; det_rd_seed = 20 # Random seeds
        obj_weights = dict(zip(objectives,w))
        metric = "".join([k*v for k,v in obj_weights.items()])

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        states[1] = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        while not done:

            ''' Inventory Routing Problem '''
            m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)

            if metric == "costs":
                action, la_decisions[day] = pIRPgym.Inventory.optimize_costs("land", m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)
            else:
                action, la_decisions[day] = pIRPgym.Inventory.optimize_environmental_indicator(metric, m, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=False)
            
            day += 1

        if save:
            other_path = True
            verbose_module.export_results.export_instance_parameters(w, seed_ix, inst_gen, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_actions(w, seed_ix, actions, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_lookahead_decisions(w, seed_ix, la_decisions, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_inventory(w, seed_ix, states, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_rewards(w, seed_ix, rewards, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_backorders(w, seed_ix, backo, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_perished(w, seed_ix, perished, other_path, theta=theta, gamma=gamma, min_q=rr)
            verbose_module.export_results.export_norm_matrix(w, seed_ix, norm_matrix, other_path, theta=theta, gamma=gamma, min_q=rr)
    
    print(f"Done seed {stoch_seed}")
    seed_ix += 1

# Verifying that the env. indicators don't matter to correct the costs action

In [ ]:
T = 7; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; theta = 0.95; gamma = 1

env_config = {'M':5,'K':2,'T':T,'F':2,'Q':375,'S':3,'LA_horizon':4,'d_max':2000,'hist_window':80, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [4]:
seed_ix = 0
for stoch_seed in feas_seeds:
    print(f"Seed {seed_ix}")
    for metric in objectives[1:]:
        rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

        stoch_rd_seed = stoch_seed; det_rd_seed = 20 # Random seeds

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        states[1] = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        while not done:

            ''' Inventory Routing Problem '''
            m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)

            print(f"\t Day {day}")
            action, la_decisions[day] = pIRPgym.Inventory.optimize_costs(metric, m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
            states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)

            day += 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07


# Random seeds search

In [3]:
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]; seed_ix = 0
objectives = ["costs","climate","water","land","fossil"]

seeds = [randint(1000,92649163) for ix in range(200)]; feas_seeds = list(); ix = 0

while len(feas_seeds) <= 50:

    stoch_seed = seeds[ix]
    try:

        for w in weights:

            rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()

            stoch_rd_seed = stoch_seed; det_rd_seed = 20 # Random seeds
            obj_weights = dict(zip(objectives,w))
            metric = "".join([k*v for k,v in obj_weights.items()])

            # Creating instance generator object
            inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
            inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

            # Resetting the environment
            states[1] = env.reset(inst_gen, return_state=True)

            done = False; day = 1
            while not done:

                ''' Inventory Routing Problem '''
                m, variables, costs, impacts = pIRPgym.Inventory.IRP.build_model(states[day], env, inst_gen)

                if metric == "costs":
                    action, la_decisions[day] = pIRPgym.Inventory.optimize_costs("land", m, costs, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                    states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=True)
                else:
                    action, la_decisions[day] = pIRPgym.Inventory.optimize_environmental_indicator(metric, m, impacts, states[day], env, inst_gen, verbose = verbose, action = True)
                    states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg, environmental=False)
                
                day += 1
        
        feas_seeds.append(stoch_seed)
        print(F"FOUND FEASIBLE SEED NUMBER {len(feas_seeds)} - {stoch_seed}")

    except:

        print(f"Failed seed at attempt {ix}")
    
    ix += 1

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "wb")
dump(feas_seeds, file); file.close()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
Failed seed at attempt 0
Failed seed at attempt 1
FOUND FEASIBLE SEED NUMBER 1 - 43558063
FOUND FEASIBLE SEED NUMBER 2 - 81483930
FOUND FEASIBLE SEED NUMBER 3 - 41646769
Failed seed at attempt 5
Failed seed at attempt 6
Failed seed at attempt 7
FOUND FEASIBLE SEED NUMBER 4 - 54110887
Failed seed at attempt 9
FOUND FEASIBLE SEED NUMBER 5 - 30305414
Failed seed at attempt 11
Failed seed at attempt 12
FOUND FEASIBLE SEED NUMBER 6 - 30719323
FOUND FEASIBLE SEED NUMBER 7 - 33721309
Failed seed at attempt 15
FOUND FEASIBLE SEED NUMBER 8 - 83494707
FOUND FEASIBLE SEED NUMBER 9 - 77892474
FOUND FEASIBLE SEED NUMBER 10 - 83092599
FOUND FEASIBLE SEED NUMBER 11 - 7052525
Failed seed at attempt 20
Failed seed at attempt 21
FOUND FEASIBLE SEED NUMBER 12 - 75457310
FOUND FEASIBLE SEED NUMBER 13 - 65013322
FOUND FEASIBLE SEED NUMBER 14 - 38781420
FOUND FEASIBLE SEED NUMBER 15 - 29073138
Failed seed at attempt 2